In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

print("loaded libraries")
spark = SparkSession.builder \
        .appName("worst vitals") \
        .getOrCreate()


loaded libraries


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 18:04:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/20 18:04:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
vitals = spark.read.parquet("/project2/wparker/SIPA_data/compressed/RCLIF_vitals_10102023.parquet")
vitals = vitals.withColumn('measured_time',f.to_timestamp('recorded_time','yyyy-MM-dd HH:mm:ss'))
vitals = vitals.select('C19_HAR_ID', 'measured_time','vital_name', 'vital_value')

In [4]:
cohort = spark.read.option("header",True).csv('/project2/wparker/SIPA_data/life_support_cohort.csv')
cohort = cohort.withColumn('life_support_start_time',f.to_timestamp('life_support_start','yyyy-MM-dd HH:mm:ss'))
cohort = cohort.select('C19_HAR_ID', 'life_support_start_time')
rows = cohort.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Rows count : 11754


In [5]:
cohort_vitals = cohort.join(vitals,'C19_HAR_ID','left')
cohort_vitals = cohort_vitals.withColumn("hour_diff", (f.col("measured_time").cast("long")-f.col("life_support_start_time").cast("long"))/(60*60))
cohort_vitals_48 = cohort_vitals.filter((f.col('hour_diff')>-42)&(f.col('hour_diff')<=5))


In [6]:
cohort_vitals_48 = cohort_vitals_48.withColumn('meas_hour', f.hour(f.col('measured_time')))
cohort_vitals_48 = cohort_vitals_48.withColumn('meas_date', f.to_date(f.col('measured_time')))
cohort_vitals_48 = cohort_vitals_48.select('C19_HAR_ID', 'life_support_start_time','meas_date',
                                           'meas_hour', 'vital_name', 'vital_value')

In [7]:
group_cols = ["C19_HAR_ID", "life_support_start_time","meas_date", "meas_hour"]
cohort_vitals_wide = cohort_vitals_48.groupBy(group_cols) \
                                     .pivot("vital_name") \
                                     .agg(f.min('vital_value').alias("min"),
                                         f.max('vital_value').alias("max")).orderBy(group_cols)

In [7]:
cohort_vitals_wide.write.parquet("/project2/wparker/SIPA_data/cohort_vitals_48_blocked.parquet", mode="overwrite")

In [9]:
group_cols = ["C19_HAR_ID", "life_support_start_time"]

cohort_vitals_wide_summary = cohort_vitals_48.groupBy(group_cols) \
                                     .pivot("vital_name") \
                                     .agg(f.min('vital_value').alias("min"),
                                         f.max('vital_value').alias("max")).orderBy("life_support_start_time")

rows = cohort_vitals_wide_summary.count()
print(f"DataFrame Rows count : {rows}")

cohort_vitals_wide_summary.write.parquet("/project2/wparker/SIPA_data/cohort_vitals_48_summary.parquet", 
                                         mode="overwrite")

DataFrame Rows count : 11653
